In [ ]:
from search import *
from game import *
from util import *
from pacman import *
import util, layout
import sys, types, time, random, os
import numpy as np



In [ ]:
# tiny genetic programming by © moshe sipper, www.moshesipper.com
from random import random, randint, seed
from statistics import mean
from copy import deepcopy

POP_SIZE        = 60   # population size
MIN_DEPTH       = 2    # minimal initial random tree depth
MAX_DEPTH       = 5    # maximal initial random tree depth
GENERATIONS     = 250  # maximal number of generations to run evolution
TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
XO_RATE         = 0.8  # crossover rate 
PROB_MUTATION   = 0.2  # per-node mutation probability 



def add(x, y): return x + y
def sub(x, y): return x - y
def mul(x, y): return x * y

"FUNCOES ( sao as operacoes executadas nos nós das arvores)-> add, sub, mul, div (x/0 = 1)*"
"TERMINAIS ( sao os tipos possiveis de nohs folha )"
'''
    TERMINAIS ( sao os tipos possiveis de nohs folha )
        -> GAMESTATE_X, onde 0 <= X < Num. de variaveis de estado do jogo
        -> CONSTANTS ( Uns numeros constantes para algumas operações, para facilitar o calculo numerico (?) )
        
    
'''




FUNCTIONS = [add, sub, mul]
TERMINALS = ['actionScore', 'eatFood', 'minGhostDist', 'distToNextPill', -1.0, 0.5, 1.0, 2.0] # 0.0,  -0.5,

STATE_VARS = ['actionScore', 'eatFood', 'minGhostDist', 'distToNextPill']
" actionScore: scoreFuturo - scoreAtual "
" eatFood: numFoodAtual - numFoodFuturo"
" minGhostDist: sqrt((pacmanY - pacmanX)**2 + (ghostY - ghost)X**2) "
" distToNextPill: sqrt((pacmanY - pacmanX)**2 + (ghostY - ghost)X**2) ?????????? "


class GPTree:
    def __init__(self, data = None, left = None, right = None):
        self.data  = data
        self.left  = left
        self.right = right
        
    def setLeft(self, tree):
        self.left = tree
        
    def setRight(self, tree):
        self.right = tree
        
    def node_label(self): # string label
        if (self.data in FUNCTIONS):
            return self.data.__name__
        else: 
            return str(self.data)
    
    def print_tree(self, prefix = ""): # textual printout
        print("%s%s" % (prefix, self.node_label()))        
        if self.left:  self.left.print_tree (prefix + "   ")
        if self.right: self.right.print_tree(prefix + "   ")

    def computeTree(self, stateVars): 
        if (self.data in FUNCTIONS): 
            return self.data(self.left.computeTree(stateVars), self.right.computeTree(stateVars))
        elif (self.data in STATE_VARS): 
            return stateVars[self.data]
        else: return self.data
        
    def size(self): # tree size in nodes
        if self.data in TERMINALS: return 1
        l = self.left.size()  if self.left  else 0
        r = self.right.size() if self.right else 0
        return 1 + l + r
    
    def random_tree(self, grow, max_depth, depth = 0): # create random tree using either grow or full method
        if depth < MIN_DEPTH or (depth < max_depth and not grow): 
            self.data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
        elif depth >= max_depth:   
            self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
        else: # intermediate depth, grow
            if random () > 0.5: 
                self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
            else:
                self.data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
        if self.data in FUNCTIONS:
            self.left = GPTree()          
            self.left.random_tree(grow, max_depth, depth = depth + 1)            
            self.right = GPTree()
            self.right.random_tree(grow, max_depth, depth = depth + 1)
            
    def getSubtree(self):
        t = GPTree()
        t.data = self.data
        if self.left:  t.left  = self.left.getSubtree()
        if self.right: t.right = self.right.getSubtree()
        return t
    
    def scan_tree(self, count, second): # note: count is list, so it's passed "by reference"
        count[0] -= 1            
        if count[0] <= 1: 
            if not second: # return subtree rooted here
                return self.getSubtree()
            else: # glue subtree here
                self.data  = second.data
                self.left  = second.left
                self.right = second.right
        else:  
            ret = None              
            if self.left  and count[0] > 1: ret = self.left.scan_tree(count, second)  
            if self.right and count[0] > 1: ret = self.right.scan_tree(count, second)  
            return ret
        

In [ ]:
tree = GPTree()
tree.random_tree(True, 2)
tree.print_tree()
thisdict = {
    STATE_VARS[0]: 1.0,
    STATE_VARS[1]: 2.0,
    STATE_VARS[2]: 3.0,
    STATE_VARS[3]: 4.0,
}
print(tree.computeTree(thisdict))

In [ ]:
" Criar populacao inicial"
''' Para a criação da população inicial, utilizamos o método RandomTree() que gera árvores randomicamente dentro 
    do domínio de operações fundamentais (add, sub, mul) 
        e dos terminais definidos (variaveis de estado e constantes)
'''

In [ ]:
POP_SIZE = 32
N_TRAIN = 4


def initialPopulation(populationSize=32,treeDepth=3):
    population = []
    for _ in range(populationSize):
        tree = GPTree()
        tree.random_tree(True, treeDepth)
        population.append(tree)
    return population

In [ ]:
" FITNESS "

# stateVars -> dictionary  ex stateVars['distX'] = distXGameStateVariable
def getFitness(individuo):
    return trainIndividual(individuo)

def getPopFitness(population):
    return np.array([getFitness(population[i]) for i in range(0,len(population))])

    
    

" SELEÇÂO "
'''Para algotimo de seleção, adotaremos inicialmente o método de seleção por Torneio, na ideia de
    aumentar as randomicidade do modelo.
'''
def tournament(population, fitness):
    indexes = np.random.choice(range(0,len(population)),2,False)
    chosen = [fitness[i] for i in indexes]
    return population[indexes[chosen.index(max(chosen))]]

def getParent(population, fitness):
    return tournament(population, fitness)


" CROSSOVER "
''' Para a reprodução dos pais, utilizamos uma função de crossover para árvores, 
'''

def crossover(treeA, treeB): # xo 2 trees at random nodes
#     if random() < XO_RATE:
#         geneA = treeA.scan_tree([randint(1, treeA.size())], None) # 1st random subtree
#         geneB = treeB.scan_tree([randint(1, treeB.size())], None) # 2nd random subtree
#         treeA.scan_tree([randint(1, treeA.size())], geneB) # 2nd subtree "glued" inside 1st tree
#         treeB.scan_tree([randint(1, treeB.size())], geneA) # 1st subtree "glued" inside 2nd tree
        
    child1 = treeA.getSubtree()
    child2 = treeB.getSubtree()
    
    aux = child1.right.getSubtree()
    aux.print_tree()
    child1.right = child2.left.getSubtree()
    child2.left = aux
    
    return child1, child2
    
            
" Loop de Reproducao"


In [ ]:
tree = GPTree()
tree.random_tree(True, 3)
tree.print_tree()

tree2 = GPTree()
tree2.random_tree(True, 3)
tree2.print_tree()

c1, c2 = crossover(tree, tree2)
c1.print_tree()
c2.print_tree()

In [ ]:
n = Directions.NORTH
s = Directions.SOUTH
e = Directions.EAST
w = Directions.WEST

def manhattanDistance(posA, posB):
    return (abs(posB[0] - posA[0]) + abs(posB[1] - posA[1]))

def calcMinGhostDist(pacmanPosition, ghostsPosition):
    dist = 99999
    for ghost in ghostsPosition:
        if dist > manhattanDistance(pacmanPosition, ghost):
            dist = manhattanDistance(pacmanPosition, ghost)
            
    return dist

def manhattanFoodDistance(posA, posX, posY):
    return (abs(posX - posA[0]) + abs(posY - posA[1]))

def calcMinFoodDist(pacmanPosition, foodGrid):
    dist = 99999
    for i in range(len(foodGrid[:])):
        for j in range(len(foodGrid[0][:])):
            if foodGrid[i][j] and (dist > manhattanFoodDistance(pacmanPosition, i, j)):
                dist = manhattanFoodDistance(pacmanPosition, i, j)

    return dist

class MyGoWestAgent(Agent):
    "An agent that goes West until it can't."
    def __init__(self, tree = None):
        self.tree = tree
        
    def setTree(self, tree):
        selt.tree = tree

    def getAction(self, state):
        "The agent receives a GameState (defined in pacman.py)."
        print(state.getLegalPacmanActions())
        print(state.getPacmanState())

        print(state.getPacmanPosition())
        print(state.getNumFood())
        print(state.getScore())
        count = 0
        legalActions = state.getLegalPacmanActions()
        legalActions.remove(Directions.STOP) #remove a opcao Stop
        actionScore = []
        stateVars = {}
        for action in legalActions:
            futureState = state.generatePacmanSuccessor(action)
            futureScore = futureState.getScore() - state.getScore()
            eatFood = state.getNumFood() - futureState.getNumFood()
            ghostDist = calcMinGhostDist(futureState.getPacmanPosition(),futureState.getGhostPositions())
            foodDist = calcMinFoodDist(futureState.getPacmanPosition(),futureState.getFood())
            stateVars[STATE_VARS[0]] = futureScore
            stateVars[STATE_VARS[1]] = eatFood
            stateVars[STATE_VARS[2]] = ghostDist
            stateVars[STATE_VARS[3]] = foodDist

            treeValue = tree.computeTree(stateVars)
            actionScore.append(treeValue)
            
        
            
        maxIndex = actionScore.index(max(actionScore))
        print("Action Score: ",actionScore)
        action = legalActions[maxIndex]
        
        futureState = state.generatePacmanSuccessor(action)
        print("MinGhostDistance: ", calcMinGhostDist(futureState.getPacmanPosition(),futureState.getGhostPositions()))
        if action in state.getLegalPacmanActions():
            return action
        else:
            return Directions.STOP

In [ ]:
def trainIndividual(ind, layoutType='smallClassic', numGames=1):
    gameLayout = layout.getLayout( 'smallClassic' )
    print("------------------------------------------ train --------------------")
    ind.print_tree()

    pacman = MyGoWestAgent(ind)

    ghostType = loadAgent('RandomGhost', True)
    ghosts = [ghostType( i+1 ) for i in range( 4 )]

    import textDisplay
    textDisplay.SLEEP_TIME = 0.1
    gameDisplay = textDisplay.PacmanGraphics()

    games = runGames( gameLayout, pacman, ghosts, gameDisplay, numGames, False)
    scores = [game.state.getScore() for game in games]
    wins = [game.state.isWin() for game in games]
    winRate = wins.count(True)/ float(len(wins))
    avgScore = sum(scores) / float(len(scores))
    return avgScore

In [ ]:
def generationLoop():
    population = initialPopulation(6)
#     fitness = np.zeros((1,len(population)))
    count = 1
    print("G - ",count,"    ============================================  ")
    while count <= 3:
        # train population to get fitness
        
        fitness = getPopFitness(population)
        print( fitness )
        
        children = []
        
        for i in range(0,int(len(population)/2)):
            father = getParent(population, fitness)
            mother = getParent(population, fitness)
            
            child1, child2 = crossover(father, mother)
            children.append(child1)
            children.append(child2)
            

#         children = mutation(children,int(len(children)*0.75))#mutation

        #elitism
#         fittest_parent = population[np.argmax(fitness)]
#         children[np.argmin(getFitness(children))] = fittest_parent

        population = children[:]
        print(children)
        
        count += 1
        
    return population

In [ ]:
population = generationLoop()
for p in population:
    p.print_tree()
    print(" - - - - ")

In [ ]:
# args = readCommand(['--layout', 'smallClassic', '--pacman', 'GoWestAgent', '-t']) # Get game components based on input
gameLayout = layout.getLayout( 'smallClassic' )

ind = GPTree()
ind.random_tree(True, 3)
ind.print_tree()

pacman = MyGoWestAgent(ind)
# pacman.setTree(ind)

ghostType = loadAgent('RandomGhost', True)
ghosts = [ghostType( i+1 ) for i in range( 4 )]

import textDisplay
textDisplay.SLEEP_TIME = 0.1
gameDisplay = textDisplay.PacmanGraphics()

runGames( gameLayout, pacman, ghosts, gameDisplay, 4, False)